# Analyze Startup Investments from Crunchbase.com
We will use techniques in pandas to analyze a large dataset. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

We will practice working with different memory constraints. In this step, we will assume we only have 10 megabytes of available memory. While crunchbase-investments.csv consumes 10.3 megabytes of disk space, pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns). To be on the safe side we will read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.

## Determine The Optimal Chunk Size

In [1]:
import pandas as pd
import sqlite3

In [2]:
# mem_avail is the memory in MB available for processing a chunk of data
# mem_thres is the acceptable memeory footprint of the chunk of data. In our case it is considered to be less than 50% of the mem_avail
def optimum_rows_in_chunk (mem_avail):
    fname = 'crunchbase-investments.csv'
    mem_thres = .5*mem_avail
    chunk_iter = pd.read_csv(fname, chunksize=5000)
    mem_usages = []
    for chunk in chunk_iter:
        mem = chunk.memory_usage(deep=True).sum()/(1024**2)
        mem_usages.append(round(mem,2))
    #print('The memory usages of the 5000 rows-chunks:\n{}'.format(mem_usages))

    print("{:8}{:^20}".format("","Memory (MB)"))
    for ind, val in enumerate(mem_usages): 
        print("chunk-{:<2}\t{:<10.2f}".format(ind+1, val))

    print('The average memory usage is {:.2f} MB'.format(sum(mem_usages)/len(mem_usages)))

In [3]:
# Check if the memory usage of 5000 rows is within 10 MB of memory
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)
mem_usages = []
for chunk in chunk_iter:
    mem = chunk.memory_usage(deep=True).sum()/(1024**2)
    mem_usages.append(round(mem,2))
#print('The memory usages of the 5000 rows-chunks:\n{}'.format(mem_usages))

print("{:8}{:^20}".format("","Memory (MB)"))
for ind, val in enumerate(mem_usages): 
    print("chunk-{:<2}\t{:<10.2f}".format(ind+1, val))
    
print('The average memory usage is {:.2f} MB'.format(sum(mem_usages)/len(mem_usages)))
    

            Memory (MB)     
chunk-1 	5.58      
chunk-2 	5.53      
chunk-3 	5.54      
chunk-4 	5.53      
chunk-5 	5.52      
chunk-6 	5.55      
chunk-7 	5.53      
chunk-8 	5.51      
chunk-9 	5.40      
chunk-10	4.64      
chunk-11	2.66      
The average memory usage is 5.18 MB


### Comment
It is safe to read 5000 rows in a chunk since the average memory usage is 5.18 MB which is almost 50% of the available 10 MB memory.

## Missing Values In The Fields

In [4]:
#Missing values of each column
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)
chunks_missing_vals = []
total_len = 0
for chunk in chunk_iter:
    chunks_missing_vals.append(chunk.isnull().sum())
    total_len += chunk.shape[0]
net_missing_vals = pd.concat(chunks_missing_vals)
total_missing_vals = net_missing_vals.groupby(net_missing_vals.index).sum()
sorted_tot_miss_vals = total_missing_vals.sort_values()
percentage_missing_vals = sorted_tot_miss_vals*100/total_len
print("{:<25}{:<5}\t{:<5}".format("Column Name","Missing Count","Percentage"))
for ind in range(len(sorted_tot_miss_vals)):
    print("{:<30}{:<5}\t{:>10.2f}".format(sorted_tot_miss_vals.index[ind], sorted_tot_miss_vals.iloc[ind], percentage_missing_vals.iloc[ind]))

Column Name              Missing Count	Percentage
company_country_code          1    	      0.00
company_name                  1    	      0.00
company_permalink             1    	      0.00
company_region                1    	      0.00
investor_region               2    	      0.00
investor_permalink            2    	      0.00
investor_name                 2    	      0.00
funded_quarter                3    	      0.01
funded_at                     3    	      0.01
funded_month                  3    	      0.01
funded_year                   3    	      0.01
funding_round_type            3    	      0.01
company_state_code            492  	      0.93
company_city                  533  	      1.01
company_category_code         643  	      1.22
raised_amount_usd             3599 	      6.81
investor_country_code         12001	     22.70
investor_city                 12480	     23.61
investor_state_code           16809	     31.79
investor_category_code        50427	     95.38


### Comment
- Columns with more than 50% missing values can be dropped from further analysis. In this data the investor_category_code has 95% of its values missing hence will be dropped.
- All the columns have at least one missing value. 


## Memory Footprint Of Each Field and The Dataset

In [5]:
# Memory footprint of each column and Total memory of the dataset
chunks_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)
chunks_mem_usage = []
for chunk in chunks_iter:
    chunks_mem_usage.append(chunk.memory_usage(deep=True))
chunks_mem_usage_combined = pd.concat(chunks_mem_usage)
total_chunks_mem_usage = chunks_mem_usage_combined.groupby(chunks_mem_usage_combined.index).sum()/(1024**2)
total_chunks_mem_usage_desc = total_chunks_mem_usage.sort_values(ascending=False)
print("{:<25}{:<5}".format("Column Name","Memory (MB)"))
for ind in range(len(total_chunks_mem_usage_desc)):
    print("{:<30}{:<10.2f}".format(total_chunks_mem_usage_desc.index[ind], total_chunks_mem_usage_desc.iloc[ind]))
print("Net memory footprint of the dataset is {:6.2f} MB.".format(total_chunks_mem_usage_desc.sum()))

Column Name              Memory (MB)
investor_permalink            4.75      
company_permalink             3.87      
investor_name                 3.74      
company_name                  3.43      
funded_at                     3.38      
company_city                  3.34      
company_category_code         3.26      
company_region                3.25      
funding_round_type            3.25      
investor_region               3.24      
funded_quarter                3.23      
funded_month                  3.23      
company_country_code          3.03      
company_state_code            2.96      
investor_city                 2.75      
investor_country_code         2.52      
investor_state_code           2.36      
investor_category_code        0.59      
raised_amount_usd             0.40      
funded_year                   0.40      
Index                         0.00      
Net memory footprint of the dataset is  56.99 MB.


# Explore the column types

Before adding the data into sqlite, we will identify the types for each column.

In [6]:
#Data structure of the first chunk
chunks_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)
for chunk in chunks_iter:
    print(chunk.info())
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

We will check for any inconsitancy in data type across the chunks for each column.

In [7]:
#Determine if data type of each column is consitant across the chunks.
chunks_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)
col_dtypes_chunks = {}
for idx, chunk in enumerate(chunks_iter):
    chunkname = "chunk{:<2}".format(idx+1)
    col_dtypes_chunks[chunkname] = chunk.dtypes
print(col_dtypes_chunks)        

{'chunk1 ': company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object, 'chunk2 ': company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_n

In [8]:
#Determine the columns that conserve/do not conserve datatype across the chunks
print(pd.DataFrame(col_dtypes_chunks))

                        chunk1   chunk2   chunk3   chunk4   chunk5   chunk6   \
company_permalink        object   object   object   object   object   object   
company_name             object   object   object   object   object   object   
company_category_code    object   object   object   object   object   object   
company_country_code     object   object   object   object   object   object   
company_state_code       object   object   object   object   object   object   
company_region           object   object   object   object   object   object   
company_city             object   object   object   object   object   object   
investor_permalink       object   object   object   object   object   object   
investor_name            object   object   object   object   object   object   
investor_category_code   object  float64  float64  float64  float64  float64   
investor_country_code    object   object   object   object   object   object   
investor_state_code      object   object

### Comment
Columns with mixed data types are 
- investor_category_code has object and float64
- investor_country_code has object and float64
- investor_state_code has object and float64
- investor_city has object and float64
- funded_year has int64 and float64

In [9]:
#Overview of the dataset
initial_10 = pd.read_csv('crunchbase-investments.csv', nrows=10)
print(initial_10)

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   
5       /company/tackk        Tackk                   web   
6    /company/acclaimd     Acclaimd             analytics   
7    /company/acclaimd     Acclaimd             analytics   
8    /company/toviefor     ToVieFor             ecommerce   
9    /company/ohk-labs     OHK Labs                sports   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               

## Investor_category_code

In [10]:
# Explore the mixed-data-type of investor_category_code
# Print the inital 10 rows from chunk1 and chunk2
chunks_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=['investor_category_code'])
chunk_no = 1
for chunk in chunks_iter:
    if chunk_no <= 2:
     print(chunk.head(10))
    chunk_no += 1

  investor_category_code
0                    NaN
1                finance
2                finance
3                finance
4                finance
5                finance
6                finance
7                finance
8                    NaN
9                    NaN
      investor_category_code
5000                     NaN
5001                     NaN
5002                     NaN
5003                     NaN
5004                     NaN
5005                     NaN
5006                     NaN
5007                     NaN
5008                     NaN
5009                     NaN


### Comment
We found earlier that the investor_category_code has approx. 95% null values. The null values are represented as NAN from the float type. Except the few data that belongs to finance, the rest of the values are null of type float64. This column is ignored as 95% of null values will make it not useful for any data analysis.

In [11]:
# Explore the distribution of values in investor_category_code
import numpy as np
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, usecols=['investor_category_code'])
value_counts_chunks = []
total_data = 0
for chunk in chunk_iter:
    value_counts_chunks.append(chunk['investor_category_code'].value_counts(dropna=False))
    total_data += chunk.shape[0]
overall_value_counts = pd.concat(value_counts_chunks)
sorted_overall_value_counts = overall_value_counts.groupby(overall_value_counts.index, dropna=False).sum().sort_values(ascending=False)
normalized_overall_value_counts = sorted_overall_value_counts*100/total_data
print("{:20}{:8}{:8}".format("Values", "Count", "Percentage"))
for indx,val in enumerate(normalized_overall_value_counts):
        print("{:<20}{:<8}{:<8}".format(normalized_overall_value_counts.index[indx], sorted_overall_value_counts.iloc[indx], round(val,2))) 


Values              Count   Percentage
nan                 50427   95.38   
finance             586     1.11    
other               371     0.7     
software            212     0.4     
web                 174     0.33    
biotech             161     0.3     
enterprise          111     0.21    
network_hosting     90      0.17    
hardware            84      0.16    
social              82      0.16    
public_relations    80      0.15    
mobile              72      0.14    
consulting          66      0.12    
games_video         51      0.1     
ecommerce           46      0.09    
health              41      0.08    
semiconductor       39      0.07    
cleantech           34      0.06    
security            32      0.06    
news                30      0.06    
search              27      0.05    
advertising         22      0.04    
messaging           7       0.01    
education           6       0.01    
legal               4       0.01    
government          3       0.01    

In [12]:
# Define function to explore the distribution of values in a column
def dist_values(filename, chunk_size, colname):
    import numpy as np
    chunk_iter = pd.read_csv(filename, chunksize=chunk_size, usecols=[colname])
    value_counts_chunks = []
    total_data = 0
    for chunk in chunk_iter:
        value_counts_chunks.append(chunk[colname].value_counts(dropna=False))
        total_data += chunk.shape[0]
    overall_value_counts = pd.concat(value_counts_chunks)
    sorted_overall_value_counts = overall_value_counts.groupby(overall_value_counts.index, dropna=False).sum().sort_values(ascending=False)
    normalized_overall_value_counts = sorted_overall_value_counts*100/total_data
    print("{:20}{:8}{:8}".format("Values", "Count", "Percentage"))
    for indx,val in enumerate(normalized_overall_value_counts):
            print("{:<20}{:<8}{:<8}".format(normalized_overall_value_counts.index[indx], sorted_overall_value_counts.iloc[indx], round(val,2))) 


In [13]:
dist_values('crunchbase-investments.csv', 5000, 'investor_category_code')

Values              Count   Percentage
nan                 50427   95.38   
finance             586     1.11    
other               371     0.7     
software            212     0.4     
web                 174     0.33    
biotech             161     0.3     
enterprise          111     0.21    
network_hosting     90      0.17    
hardware            84      0.16    
social              82      0.16    
public_relations    80      0.15    
mobile              72      0.14    
consulting          66      0.12    
games_video         51      0.1     
ecommerce           46      0.09    
health              41      0.08    
semiconductor       39      0.07    
cleantech           34      0.06    
security            32      0.06    
news                30      0.06    
search              27      0.05    
advertising         22      0.04    
messaging           7       0.01    
education           6       0.01    
legal               4       0.01    
government          3       0.01    

## Investor_county_code

In [14]:
# Explore the mixed-data-type of investor_country_code
# Print the inital 10 rows from chunk1, chunk8, chunk9 and chunk10
chunks_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=['investor_country_code'])
chunk_no = 1
for chunk in chunks_iter:
    if chunk_no in [1, 8, 9, 10]:
     print(chunk.head(10))
    chunk_no += 1

  investor_country_code
0                   USA
1                   USA
2                   USA
3                   USA
4                   USA
5                   USA
6                   USA
7                   USA
8                   NaN
9                   NaN
      investor_country_code
35000                   USA
35001                   USA
35002                   USA
35003                   USA
35004                   USA
35005                   USA
35006                   USA
35007                   USA
35008                   USA
35009                   USA
      investor_country_code
40000                   USA
40001                   USA
40002                   USA
40003                   USA
40004                   USA
40005                   USA
40006                   USA
40007                   USA
40008                   USA
40009                   USA
       investor_country_code
45000                    NaN
45001                    NaN
45002                    NaN
4500

### Comment
Investor_country_code has approx. 23% null values. The null values are NAN from the float type. The null values are leading to inconsistent data type along the chunks. 

In [15]:
# Distribution of values in the Investor_country_code
dist_values('crunchbase-investments.csv', 5000, 'investor_country_code')

Values              Count   Percentage
USA                 36574   69.18   
nan                 12001   22.7    
GBR                 825     1.56    
ISR                 521     0.99    
CAN                 432     0.82    
DEU                 349     0.66    
FRA                 229     0.43    
JPN                 222     0.42    
CHE                 184     0.35    
AUS                 163     0.31    
CHN                 143     0.27    
SWE                 120     0.23    
IND                 105     0.2     
SGP                 105     0.2     
KOR                 74      0.14    
NLD                 72      0.14    
DNK                 70      0.13    
RUS                 63      0.12    
HKG                 53      0.1     
IRL                 50      0.09    
TWN                 47      0.09    
ESP                 45      0.09    
BEL                 44      0.08    
NOR                 37      0.07    
FIN                 36      0.07    
LUX                 32      0.06    

# Investor_state_code

In [16]:
# Distribution of values in the Investor_state_code
dist_values('crunchbase-investments.csv', 5000, 'investor_state_code')

Values              Count   Percentage
CA                  18405   34.81   
nan                 16809   31.79   
NY                  4404    8.33    
MA                  3619    6.85    
IL                  992     1.88    
WA                  847     1.6     
TX                  816     1.54    
PA                  762     1.44    
CO                  729     1.38    
VA                  579     1.1     
CT                  577     1.09    
MD                  486     0.92    
NJ                  456     0.86    
NC                  339     0.64    
DC                  323     0.61    
MI                  315     0.6     
OH                  309     0.58    
GA                  274     0.52    
FL                  242     0.46    
UT                  200     0.38    
MO                  148     0.28    
TN                  147     0.28    
MN                  101     0.19    
RI                  92      0.17    
IN                  88      0.17    
OR                  85      0.16    

### Comment
The presence of 32% null values is leading to mixed datatypes when chunks of 5000 rows are read for data analysis.

# Investor_city

In [17]:
# Distribution of values in the Investor_city
dist_values('crunchbase-investments.csv', 5000, 'investor_city')

Values              Count   Percentage
nan                 12480   23.61   
Menlo Park          7086    13.4    
New York            3899    7.37    
Palo Alto           3224    6.1     
San Francisco       3124    5.91    
Boston              1702    3.22    
Cambridge           972     1.84    
Chicago             742     1.4     
Mountain View       724     1.37    
London              699     1.32    
Santa Clara         606     1.15    
Seattle             604     1.14    
Waltham             528     1.0     
Boulder             516     0.98    
Austin              356     0.67    
Santa Monica        335     0.63    
Washington          302     0.57    
San Mateo           276     0.52    
San Diego           273     0.52    
Los Angeles         262     0.5     
San Jose            247     0.47    
Dallas              241     0.46    
Atlanta             238     0.45    
Durham              214     0.4     
Greenwich           211     0.4     
Redwood City        205     0.39    

Pune                1       0.0     
Pullach             1       0.0     
Kranj               1       0.0     
LYON                1       0.0     
Lake Mary           1       0.0     
Preston             1       0.0     
Lake Success        1       0.0     
Lancaster           1       0.0     
Larkspur            1       0.0     
Leawood             1       0.0     
Leeds               1       0.0     
Lenexa              1       0.0     
Marietta            1       0.0     
Marlborough         1       0.0     
Martinsried / Munich1       0.0     
Overland Park       1       0.0     
Mississauga, Ontario1       0.0     
Norfolk             1       0.0     
Noida               1       0.0     
Mkalles             1       0.0     
Ningbo              1       0.0     
Monroe Township     1       0.0     
Mont-Saint-Guibert  1       0.0     
Newport Coast       1       0.0     
Montgomery          1       0.0     
Montreal (Quebec)   1       0.0     
Montr̩al (Qu̩bec)   1       0.0     
M

### Comment
The presence of 23% null values is causing inconsistent data types in investor_city column.

# Funded_year

In [18]:
# Distribution of values in the funded_year
dist_values('crunchbase-investments.csv', 5000, 'funded_year')

Values              Count   Percentage
2012.0              8773    16.59   
2011.0              8259    15.62   
2013.0              7115    13.46   
2010.0              6295    11.91   
2008.0              4937    9.34    
2007.0              4781    9.04    
2009.0              4710    8.91    
2006.0              3867    7.31    
2005.0              2990    5.66    
2004.0              411     0.78    
2000.0              176     0.33    
2003.0              172     0.33    
2002.0              145     0.27    
2001.0              116     0.22    
1999.0              95      0.18    
1998.0              11      0.02    
1995.0              5       0.01    
1997.0              5       0.01    
nan                 3       0.01    
1996.0              2       0.0     
1987.0              2       0.0     


### Comment 
Mixed data type int and float due to the nan that belongs to float type. 

## Load Into SQLite Database

In [19]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
for chunk in chunk_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)